<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Modelo ARIMA</h4>
</center>

In [84]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
!pip install pmdarima
from pmdarima.arima import auto_arima
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

# !pip install darts
from darts import TimeSeries
# !pip install u8darts
from darts.utils.missing_values import fill_missing_values
from darts.metrics import mape
from darts.models import TBATS, AutoARIMA, Theta
from darts import concatenate
from darts.utils.timeseries_generation import datetime_attribute_timeseries as dt_attr
from darts.models import NBEATSModel


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
df = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product.csv", header = None)
df.columns = ["periodo", "product_id", "tn"]
df.head(5)

,periodo,product_id,tn
0,2018-01-01,20001,1169.07532
1,2018-01-01,20002,984.80167
2,2018-01-01,20003,907.56304
3,2018-01-01,20004,415.52538
4,2018-01-01,20005,417.53208


- Predicción 1

In [90]:
products = df.product_id.drop_duplicates().to_list()
results = pd.DataFrame(columns=['product_id', 'prediccion_mes_2'])
for product in products:
    df_product = df[df.product_id == product]
    df_product['periodo'] = pd.to_datetime(df_product['periodo'])
    df_product = df_product.set_index('periodo')
    df_product = df_product.resample('M').mean()
    df_product.index = df_product.index.to_period('M').to_timestamp()
    df_product = df_product.dropna()
    if len(df_product) < 2:
        print(f"No hay suficientes datos para el product_id: {product}")
        continue
    try:
        model = auto_arima(df_product.tn, seasonal=False, trace=False, error_action='ignore', suppress_warnings=True)
        forecast = model.predict(n_periods=2)
        prediccion_mes_2 = np.array(forecast)[1]
        results = pd.concat([results, pd.DataFrame({'product_id': [product], 'prediccion_mes_2': [prediccion_mes_2]})])
    except Exception as e:
        print(f"Error al ajustar el modelo para el product_id {product}: {e}")

In [94]:
results.rename(columns = {"prediccion_mes_2":"tn"}).to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_arima_1.csv", sep = ",", index = False)

- Predicción 2

In [96]:
products = df.product_id.drop_duplicates().to_list()
results = pd.DataFrame(columns=['product_id', 'prediccion_mes_2'])
for product in products:
    df_product = df[df.product_id == product]
    df_product['periodo'] = pd.to_datetime(df_product['periodo'])
    df_product = df_product.set_index('periodo')
    df_product = df_product.resample('M').mean()
    df_product.index = df_product.index.to_period('M').to_timestamp()
    df_product = df_product.dropna()
    if len(df_product) < 2:
        print(f"No hay suficientes datos para el product_id: {product}")
        continue
    try:
        # Ajustar el modelo AUTOARIMA con parámetros personalizados
        model = auto_arima(df_product.tn, 
                           seasonal="auto", 
                           start_p=1, max_p=5,
                           start_q=1, max_q=5,
                           max_order=10,
                           stepwise=True,  # Utilizar enfoque de selección de hiperparámetros paso a paso
                           trace=False, 
                           error_action='ignore', 
                           suppress_warnings=True)
        forecast = model.predict(n_periods=2)
        prediccion_mes_2 = np.array(forecast)[1]
        results = pd.concat([results, pd.DataFrame({'product_id': [product], 'prediccion_mes_2': [prediccion_mes_2]})])
    except Exception as e:
        print(f"Error al ajustar el modelo para el product_id {product}: {e}")

In [97]:
results.rename(columns = {"prediccion_mes_2":"tn"}).to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_arima_2.csv", sep = ",", index = False)

- Predicción 3

In [99]:
products = df.product_id.drop_duplicates().to_list()
results = pd.DataFrame(columns=['product_id', 'prediccion_mes_2'])
for product in products:
    df_product = df[df.product_id == product]
    df_product['periodo'] = pd.to_datetime(df_product['periodo'])
    df_product = df_product.set_index('periodo')
    df_product = df_product.resample('M').mean()
    df_product.index = df_product.index.to_period('M').to_timestamp()
    df_product = df_product.dropna()
    if len(df_product) < 2:
        print(f"No hay suficientes datos para el product_id: {product}")
        continue
    try:
        # Ajustar el modelo AUTOARIMA con parámetros personalizados
        model = auto_arima(df_product.tn, 
                           seasonal="auto", 
                           d=1, max_d=5,
                           start_p=1, max_p=12,
                           start_q=1, max_q=12,
                           max_order=20,
                           stepwise=True,  # Utilizar enfoque de selección de hiperparámetros paso a paso
                           trace=False, 
                           error_action='ignore', 
                           suppress_warnings=True)
        forecast = model.predict(n_periods=2)
        prediccion_mes_2 = np.array(forecast)[1]
        results = pd.concat([results, pd.DataFrame({'product_id': [product], 'prediccion_mes_2': [prediccion_mes_2]})])
    except Exception as e:
        print(f"Error al ajustar el modelo para el product_id {product}: {e}")

In [100]:
results.rename(columns = {"prediccion_mes_2":"tn"}).to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_arima_3.csv", sep = ",", index = False)

- Metodo DARTS

Para las series que lo permite la libreria (+10 puntos historia)

In [ ]:
listado_productos = df.groupby(["product_id"])["periodo"].nunique().reset_index().sort_values(by="periodo", ascending = False)
products = listado_productos[listado_productos.periodo >= 10].product_id.drop_duplicates().to_list()

results = pd.DataFrame(columns=['product_id', 'prediccion_mes_2'])

for product in products:
    df_product = df[df['product_id'] == product]
    series = TimeSeries.from_dataframe(df_product,
                                       time_col="periodo",
                                       value_cols="tn",
                                       freq='MS')
    
    # Rellenar valores faltantes en la serie de tiempo
    series = fill_missing_values(series)
    model = AutoARIMA()
    model.fit(series)
    forecast = model.predict(n=2)
    prediccion_mes_2 = forecast.last_values()[0]
    
    results = pd.concat([results, pd.DataFrame({'product_id': [product], 'prediccion_mes_2': [prediccion_mes_2]})])

In [ ]:
another_products = listado_productos[listado_productos.periodo < 10].product_id.drop_duplicates().to_list()
results_another_products = pd.DataFrame(columns=['product_id', 'prediccion_mes_2'])
for product in another_products:
    df_product = df[df.product_id == product]
    df_product['periodo'] = pd.to_datetime(df_product['periodo'])
    df_product = df_product.set_index('periodo')
    df_product = df_product.resample('M').mean()
    df_product.index = df_product.index.to_period('M').to_timestamp()
    df_product = df_product.dropna()
    if len(df_product) < 2:
        print(f"No hay suficientes datos para el product_id: {product}")
        continue
    try:
        # Ajustar el modelo AUTOARIMA con parámetros personalizados
        model = auto_arima(df_product.tn, 
                           seasonal="auto", 
                           start_p=1, max_p=5,
                           start_q=1, max_q=5,
                           max_order=10,
                           stepwise=True,  # Utilizar enfoque de selección de hiperparámetros paso a paso
                           trace=False, 
                           error_action='ignore', 
                           suppress_warnings=True)
        forecast = model.predict(n_periods=2)
        prediccion_mes_2 = np.array(forecast)[1]
        results_another_products = pd.concat([results_another_products, pd.DataFrame({'product_id': [product], 'prediccion_mes_2': [prediccion_mes_2]})])
    except Exception as e:
        print(f"Error al ajustar el modelo para el product_id {product}: {e}")

C:\Users\usuario\AppData\Local\Temp\ipykernel_1720\2356388629.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product['periodo'] = pd.to_datetime(df_product['periodo'])
C:\Users\usuario\AppData\Local\Temp\ipykernel_1720\2356388629.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product['periodo'] = pd.to_datetime(df_product['periodo'])
C:\Users\usuario\AppData\Local\Temp\ipykernel_1720\2356388629.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [ ]:
export = pd.concat([results, results_another_products])
print(len(export))
export.rename(columns = {"prediccion_mes_2":"tn"}).to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_darts_arima.csv", sep = ",", index = False)

780
